## Run a Custom Trading Algorithm Backtest with Minute Timeseries Pricing Data

### Set up

In [ ]:
import os
import sys
import datetime
import analysis_engine.consts as ae_consts
import analysis_engine.algo as base_algo
import analysis_engine.run_algo as run_algo
import analysis_engine.plot_trading_history as plot_trading_history
import analysis_engine.build_publish_request as build_publish_request
import spylunking.log.setup_logging as log_utils


log = log_utils.build_colorized_logger(name='bt')

### Build the Algorithm Config Dictionary

In [ ]:
config_dict = {
    'name': 'backtest',
    'timeseries': 'minute',
    'trade_horizon': 5,
    'num_owned': 10,
    'buy_shares': 10,
    'balance': 5000.0,
    'commission': 6.0,
    'ticker': 'SPY',
    'algo_module_path': None,
    'algo_version': 1,
    'buy_rules': {
        'confidence': 75,
        'min_indicators': 3
    },
    'sell_rules': {
        'confidence': 75,
        'min_indicators': 3
    },
    'indicators': [
        {
            'name': 'willr_-70_-30',
            'module_path': '/opt/sa/analysis_engine/mocks/example_indicator_williamsr.py',
            'category': 'technical',
            'type': 'momentum',
            'uses_data': 'minute',
            'high': 0,
            'low': 0,
            'close': 0,
            'open': 0,
            'willr_value': 0,
            'num_points': 80,
            'buy_below': -70,
            'sell_above': -30,
            'is_buy': False,
            'is_sell': False,
            'verbose': False  # log in just this indicator
        },
        {
            'name': 'willr_-80_-20',
            'module_path': '/opt/sa/analysis_engine/mocks/example_indicator_williamsr.py',
            'category': 'technical',
            'type': 'momentum',
            'uses_data': 'minute',
            'high': 0,
            'low': 0,
            'close': 0,
            'open': 0,
            'willr_value': 0,
            'num_points': 30,
            'buy_below': -80,
            'sell_above': -20,
            'is_buy': False,
            'is_sell': False
        },
        {
            'name': 'willr_-90_-10',
            'module_path': '/opt/sa/analysis_engine/mocks/example_indicator_williamsr.py',
            'category': 'technical',
            'type': 'momentum',
            'uses_data': 'minute',
            'high': 0,
            'low': 0,
            'close': 0,
            'open': 0,
            'willr_value': 0,
            'num_points': 60,
            'buy_below': -90,
            'sell_above': -10,
            'is_buy': False,
            'is_sell': False
        },
        {
            'name': 'willr_open_-80_-20',
            'module_path': '/opt/sa/analysis_engine/mocks/example_indicator_williamsr_open.py',
            'category': 'technical',
            'type': 'momentum',
            'uses_data': 'minute',
            'high': 0,
            'low': 0,
            'close': 0,
            'open': 0,
            'willr_open_value': 0,
            'num_points': 80,
            'buy_below': -80,
            'sell_above': -20,
            'is_buy': False,
            'is_sell': False
        }
    ],
    'verbose': False,             # log in the algorithm
    'verbose_processor': False,   # log in the indicator processor
    'verbose_indicators': False,  # log all indicators
    'verbose_trading': False,     # log in the algo trading methods
    'inspect_datasets': False     # log dataset metrics - slow
}

In [ ]:
class ExampleCustomAlgo(base_algo.BaseAlgo):
    """ExampleCustomAlgo"""

    def process(self, algo_id, ticker, dataset):
        """process

        Run a custom algorithm after all the indicators
        from the ``algo_config_dict`` have been processed and all
        the number crunching is done. This allows the algorithm
        class to focus on the high-level trade execution problems
        like bid-ask spreads and opening the buy/sell trade orders.

        **How does it work?**

        The engine provides a data stream from the latest
        pricing updates stored in redis. Once new data is
        stored in redis, algorithms will be able to use
        each ``dataset`` as a chance to evaluate buy and
        sell decisions. These are your own custom logic
        for trading based off what the indicators find
        and any non-indicator data provided from within
        the ``dataset`` dictionary. Here is what the ``dataset``
        will look like when your algorithm's ``process``
        method is called (assuming you have redis running
        with actual pricing data too):

        **Dataset Dictionary Structure**

        .. code-block:: python

            {
                'id': dataset_id,
                'date': date,
                'data': {
                    'daily': pd.DataFrame([]),
                    'minute': pd.DataFrame([]),
                    'quote': pd.DataFrame([]),
                    'stats': pd.DataFrame([]),
                    'peers': pd.DataFrame([]),
                    'news1': pd.DataFrame([]),
                    'financials': pd.DataFrame([]),
                    'earnings': pd.DataFrame([]),
                    'dividends': pd.DataFrame([]),
                    'calls': pd.DataFrame([]),
                    'puts': pd.DataFrame([]),
                    'pricing': pd.DataFrame([]),
                    'news': pd.DataFrame([])
                }
            }

        .. tip:: you can also inspect these datasets by setting
            the algorithm's config dictionary key
            ``"inspect_datasets": True``

        :param algo_id: string - algo identifier label for debugging datasets
            during specific dates
        :param ticker: string - ticker
        :param dataset: a dictionary of identifiers (for debugging) and
            multiple pandas ``DataFrame`` objects.
        """
        if self.verbose:
            log.info(
                'process start - {} '
                'balance={} '
                'date={} minute={} close={} '
                'high={} low={} open={} volume={}'
                ''.format(
                    self.name,
                    self.balance,
                    self.backtest_date, self.latest_min,
                    self.latest_close, self.latest_high,
                    self.latest_low, self.latest_open,
                    self.latest_volume))

    # end of process
# end of ExampleCustomAlgo

## Customize the Algorithm's Balance and Ticker

In [ ]:
ticker = 'SPY'
balance = 10000.00
commission = 6.0

verbose_algo = False
verbose_processor = False
verbose_indicators = False
inspect_datasets = False

config_dict['ticker'] = ticker
config_dict['balance'] = balance
config_dict['commission'] = commission

if verbose_algo:
    config_dict['verbose'] = verbose_algo
if verbose_processor:
    config_dict['verbose_processor'] = verbose_processor
if verbose_indicators:
    config_dict['verbose_indicators'] = verbose_indicators
if inspect_datasets:
    config_dict['inspect_datasets'] = inspect_datasets

In [ ]:
algo_obj = ExampleCustomAlgo(
    ticker=config_dict['ticker'],
    config_dict=config_dict)

algo_res = run_algo.run_algo(
    ticker=ticker,
    algo=algo_obj,
    raise_on_err=True)

if algo_res['status'] != ae_consts.SUCCESS:
    log.error(
        'failed running algo backtest '
        '{} hit status: {} error: {}'.format(
            algo_obj.get_name(),
            ae_consts.get_status(status=algo_res['status']),
            algo_res['err']))
# if not successful

In [ ]:
log.info(
    'backtest: {} {}'.format(
        algo_obj.get_name(),
        ae_consts.get_status(status=algo_res['status'])))

trading_history_dict = algo_obj.get_history_dataset()
history_df = trading_history_dict[ticker]
if not hasattr(history_df, 'to_json'):
    log.error(
        'no history to plot')
else:
    debug = False
    history_json_file = None
    if history_json_file:
        log.info(
            'saving history to: {}'.format(
                history_json_file))
        history_df.to_json(
            history_json_file,
            orient='records',
            date_format='iso')

    log.info('plotting history')

    first_date = history_df['date'].iloc[0]
    end_date = history_df['date'].iloc[-1]
    title = (
        'Trading History {} for Algo {}\n'
        'Backtest dates from {} to {}'.format(
            ticker,
            trading_history_dict['algo_name'],
            first_date,
            end_date))
    use_xcol = 'date'
    use_as_date_format = '%d\n%b'
    if config_dict['timeseries'] == 'minute':
        use_xcol = 'minute'
        use_as_date_format = '%d %H:%M:%S\n%b'
    xlabel = 'Dates vs {} values'.format(
        trading_history_dict['algo_name'])
    ylabel = 'Algo {}\nvalues'.format(
        trading_history_dict['algo_name'])
    df_filter = (history_df['close'] > 0.01)

    # set default hloc columns:
    blue = None
    green = None
    orange = None

    red = 'close'
    blue = 'balance'

    if debug:
        for i, r in history_df.iterrows():
            log.debug('{} - {}'.format(
                r['minute'],
                r['close']))

    plot_trading_history.plot_trading_history(
        title=title,
        df=history_df,
        red=red,
        blue=blue,
        green=green,
        orange=orange,
        date_col=use_xcol,
        date_format=use_as_date_format,
        xlabel=xlabel,
        ylabel=ylabel,
        df_filter=df_filter,
        show_plot=True,
        dropna_for_all=True)